In [10]:
import pandas as pd 
import numpy as np
import cleaning_functions as cf
import legal_functions as lf
import bail_status_dict as bs

In [11]:
df1 = pd.read_csv("../data/data_1.csv")
print(df1.shape)
df2 = pd.read_csv("../data/data_2.csv")
print(df2.shape)
df3 = pd.read_csv("../data/data_3.csv")
print(df3.shape)

(843, 11)
(558, 11)
(980, 11)


In [12]:
df1 = cf.split_info(df1)
df2 = cf.split_info(df2)
df3 = cf.split_info(df3)

In [13]:
df1 = cf.drop_null_rows(df1)
df2 = cf.drop_null_rows(df2)
df3 = cf.drop_null_rows(df3)

13 rows have been dropped
26 rows have been dropped
4 rows have been dropped


In [14]:
# concatenate vertically
dfA = pd.concat([df1, df2, df3], axis=0)

# reset the index
dfA = dfA.reset_index(drop=True)

dfA.shape

(2338, 15)

### Import Offences Lookup Table, encode it, and see how many offences are covered. Drop those that aren't

In [15]:
lookup_df = pd.read_csv("../data/OffenceLookup.csv")
lookup_df.head()

,Statute,Offence Number,Offence,Election,Discharge Available,SS Available,CSO Available,Maximum (Summary)(Years),Maximum (Indictable)(Years)
0,CCC,56.1 1,Possess ID docs,Hybrid,Y,Y,Y,2,5
1,CCC,72 1,Forcible entry,Hybrid,Y,Y,Y,2,2
2,CCC,82 1,Make/possess explosives,Hybrid,Y,Y,Y,2,5
3,CCC,85 1,Use firearm in commission of offence,Indictable,N,N,N,5,14
4,CCC,85 2,Use imitation firearm in commission of offence,Indictable,N,N,N,5,14


In [16]:
lookup_df = lf.encode_offences(lookup_df)

In [17]:
# drop rows with offence codes that don't meet the criteria
dfA = cf.clean_offence_codes(dfA)

14 rows have been dropped


In [18]:
# compare df with lookup_df and drop rows that don't have an offence in the lookup table
dfA = lf.compare_dataframes(dfA, lookup_df)

Number of values in both dataframes (to keep): 2274
Number of values not in lookup df (to drop): 50
Offences that have no lookup and are being dropped: ['502 2' '771 1' '117.05 1' '11 2' '159 1' '512.3' 'CCC 811' '503 3 b'
 '117.05 4 b' '78' '111 1' '000' 'CCC 733.1 1' '486.2 2 T' '89.6 1'
 '37 1 b' '26 1 c']


Merge df1 with offences df

In [20]:
merged_df = dfA.merge(lookup_df, on = ['Offence Number'], how='left')

In [21]:
merged_df.head()

,File,Name,Bail Status,In Custody,Lawyer,Days in Court,Info,Videoconf,Offence Location,Court Location,...,Statute_x,Offence Number,Statute_y,Offence,Election,Discharge Available,SS Available,CSO Available,Maximum (Summary)(Years),Maximum (Indictable)(Years)
0,93322-9-B,"Hampton, Braiden",AOI,Y,"Reveley,",100,JIR 001 CCC 811,NaN,Mission BC,Abbotsford,...,CCC,811,CCC,Breach of Recognizance,1,0,0,0,2,2
1,95217-2-B,"Hampton, Braiden Gray",AOI,N,"Reveley,",145,ARB 001 CCC 733.1 1,NaN,Abbotsford BC,Abbotsford,...,CCC,733.1 1,CCC,Breach of Probation,1,0,0,0,2,2
2,95824-1,"Hampton, Braiden Gray",AOI,N,"Reveley,",130,ARB 001 CCC 266,NaN,Abbotsford BC,Abbotsford,...,CCC,266,CCC,Assault,1,0,0,0,2,5
3,95935-2-C,"Hampton, Braiden Gray",PPA,Y,"Reveley,",104,JIR 001 CCC 266,NaN,Mission BC,Abbotsford,...,CCC,266,CCC,Assault,1,0,0,0,2,5
4,94351-1,"Harris, Ryan Patrick",ROD,Y,"Reveley,",391,JIR 001 CDS 5 2,NaN,Mission BC,Abbotsford,...,CDS,5 2,CDS,PPT,1,0,0,0,2,25


In [22]:
# export pre-encoded df for EDA
merged_df.to_csv("pre-encoded_df.csv")

In [23]:
# encode everything for modeling

merged_df = cf.clean_lawyers(merged_df)
merged_df = cf.encode_incustody(merged_df)
merged_df = bs.map_bail(merged_df)
merged_df = cf.encode_courts(merged_df)
merged_df = cf.drop_cols(merged_df)

c:\Users\emily\OneDrive\Desktop\Data Science Bootcamp\Final Project\src\cleaning_functions.py:38: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Lawyer'] = df['Lawyer'].str.replace(r'[^\w\s]','').str.replace(r'\b\w{1,2}\b','')


KeyError: "['Statute'] not found in axis"

In [ ]:
merged_df.to_csv("../data/clean_merged_df.csv")